# Homework 8 - NLP


## Sean McWhirter

In [69]:
from nltk.metrics import *
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pprint
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob
from collections import defaultdict
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
# Read in spreadsheet with links
df = pd.read_excel(r'C:\Users\seans\Desktop\NLP\HW5\permalinks.xlsx', header=None)

#convert to list to iterate through
link_list = df[0].tolist()

# 1.)
>1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.

I am using the TextBlob lexicon from the textbook

In [71]:
#https://stackoverflow.com/questions/16333296/how-do-you-create-nested-dict-in-python
#https://docs.python.org/3/library/collections.html#collections.defaultdict

#Using TextBlob
reviews = []
sentiment = []


for i in link_list:
    page = requests.get(i)
    
    #extract the html
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Extract text we need from the tag
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip().lower()
    
    #add it to the review list
    reviews.append(span)
    
    #Blob it
    blob = TextBlob(span)
    
    #Sentiment Analysis - this funciton removes punctuation and tokenizes 
    sent = blob.sentiment
    
    #add to sentiment list
    sentiment.append(sent.polarity)




In [72]:
#make sure they are the same length--should be 100
print(len(reviews))
print(len(sentiment))

100
100


In [73]:
#It looks like the sentiment is correct for this random sample
print(reviews[11])
print(sentiment[11])

'the boys' is a fantastically written satire that was also the brainchild of the creative genius, simon pegg. karl urban was the perfect choice for butcher, with the vocabulary of negan (the walking dead) and the vigilante drive of paul kersey (death wish). there is so much material to work with that this series could continue for several seasons. kudos to seth rogen and amazon for bringing this brilliant concept to streaming media!
0.35714285714285715


In [74]:
#Create the three levels of sentiment
x = 0.05
pos_neg = []
for i in sentiment:
    if i> x:
        pos_neg.append('Positive')
    elif i < -x:
         pos_neg.append('Negative')
    else:
         pos_neg.append('Neutral')

In [75]:
#stack into matrix; add a column for the 'positive'/'negative'/'neutral' ratings
sent_matrix = np.column_stack((reviews, sentiment,pos_neg))
sent_matrix.shape

(100, 3)

In [76]:
#sanity check
print(sent_matrix[0,:])

["started watching this brilliant spin on a superhero show after dinner. now it's 4 am and i have just watched the whole thing.from the very first episode i was glued to this exciting, interesting and crazy ride."
 '0.2708333333333333' 'Positive']


In [77]:
#Count the number of positive/negative/neutral reviews

#https://stackoverflow.com/questions/60000941/counting-number-of-specific-numbers-in-the-column-of-a-numpy-array-in-python
results, counts = np.unique(sent_matrix[:,2], return_counts=True)
results_dict = dict(zip(results, counts))

### Final Resutls


In [78]:
#print the results
results_dict

{'Negative': 11, 'Neutral': 19, 'Positive': 70}

# 2.)
>2.	For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.


In [79]:
#Cluster code from HW7

#set up to remove stopwords
new_stops = list(stopwords.words('english'))

new_stops = new_stops + ['show', 'episode', 'it', 'get', ' it.', 'it\'s', 'series', 'season', 
                         'can', 'no', 've', 'watch', 'watched', 'much', 'lot', 'thus', 'no', 'need', 'needed', 'should\'ve',
                        'it,', 'can.', 'can,']
stop_words = new_stops

In [80]:
#Cluster code from HW7


#https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
from string import punctuation
output = []
counter = 1


for i in reviews:
    lst = []
    if i not in stop_words:
        lst.append(i)
    span_nostop = ' '.join(lst)
    output.append(str(span_nostop))


In [81]:
#Cluster code from HW7

output = []

#Append reviews to a list for vectorizer

for i in link_list:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip().lower()
    for i in span.split(' '):
        if i not in stop_words:
            lst.append(i)
    span_nostop = ' '.join(lst)
    
    #span_nostop = ' '.join([i for i in span if i not in stop_words])
    #tokenized = word_tokenize(span)
    #tagged = nltk.pos_tag(tokenized)
    #chunked = np.parse(tagged)
    #for subtree in chunked.subtrees():
       # if subtree.label() == 'NP':
            #lst.append(subtree)
    #span = [i for i in span if i not in string.punctuaion]
    output.append(str(span_nostop))


In [82]:
#output[0]

In [83]:
#create vectorizer
vectorizer = TfidfVectorizer(stop_words={'english'})
#Perform tf-idf on reviews
links_vect = vectorizer.fit_transform(output)
#Look at shape
links_vect.shape

(100, 2547)

In [84]:
from sklearn.cluster import KMeans
#Cluster
kmeans = KMeans(n_clusters=6, random_state=11)
clustered = kmeans.fit(links_vect)

#Get cluster labels
clusters = clustered.labels_

In [85]:
clusters

array([4, 3, 4, 4, 4, 1, 5, 4, 1, 4, 0, 2, 1, 3, 3, 1, 3, 4, 5, 4, 0, 0,
       4, 3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 2,
       3, 2, 0, 2, 0, 5, 0, 2, 2, 2, 3, 3, 3, 0, 3, 5, 3, 3, 3, 3, 0, 1,
       5, 0, 3, 5, 3, 1, 3, 0, 0, 3, 2, 3])

In [86]:
sent_matrix2 = np.column_stack((sentiment, clusters))
sent_matrix2.shape

(100, 2)

In [87]:
#Check that it worked correclty
sent_matrix2[1:5]

array([[0.6       , 3.        ],
       [0.15833333, 4.        ],
       [0.6225    , 4.        ],
       [0.40551146, 4.        ]])

In [88]:
#Mean, median, high, low for each cluster

#Create nested dictionary to store results
results_dict = {}
for i in sent_matrix2[:,1]:
    if i not in results_dict.keys():
        results_dict[i]={}



In [89]:
#check
results_dict

{4.0: {}, 3.0: {}, 1.0: {}, 5.0: {}, 0.0: {}, 2.0: {}}

In [90]:
for i in results_dict.keys():
    tmp = []
    for j in sent_matrix2:
        if j[1]==i:
            tmp.append(j[0])
            results_dict[i]['mean']=np.mean(tmp)
            results_dict[i]['median']=np.median(tmp)
            results_dict[i]['max']=np.amax(tmp)
            results_dict[i]['min']=np.amin(tmp)

### Final Results

In [91]:
results_dict

{4.0: {'mean': 0.3023495982755242,
  'median': 0.2920634920634921,
  'max': 0.6225,
  'min': -0.275},
 3.0: {'mean': 0.15000986900085223,
  'median': 0.1267543859649123,
  'max': 0.75,
  'min': -0.36666666666666664},
 1.0: {'mean': 0.1291113138994921,
  'median': 0.12463765498976767,
  'max': 0.37006802721088433,
  'min': -0.2272727272727273},
 5.0: {'mean': 0.040767873216994405,
  'median': 0.00442279942279942,
  'max': 0.38749999999999996,
  'min': -0.177536231884058},
 0.0: {'mean': 0.20188551483272887,
  'median': 0.25,
  'max': 0.5499999999999999,
  'min': -0.15863636363636358},
 2.0: {'mean': 0.09438201507639646,
  'median': 0.10964285714285715,
  'max': 0.35714285714285715,
  'min': -0.37500000000000006}}

The mean, median, max, and min polarity scores of each cluster was a little surprising to me at first.  The mean polarity scores for each cluster is positive (>.05) with the exception of cluster 5 (0.04 or 'neutral'). I had originally expected to see a mixture of positive and negative clusters, however, after tinking about it the outcome above makes sense.  The clusters were constructed based on the content of each review, and ended up clustering the different shows together instead of sentiment-based.  It makes sense that the polarity scores in each cluster would vary so greatly in terms of min and max. 

# Extra Credit

>a. Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).

In [97]:
#NP Chunker from HW5

output = {}
counter = 1

np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

for i in link_list:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    tokenized = word_tokenize(span)
    tagged = nltk.pos_tag(tokenized)
    chunked = np.parse(tagged)
    for subtree in chunked.subtrees():
        if subtree.label() == 'NP':
            lst.append(subtree)
    
    output["Review: " + str(counter)] = lst
    counter +=1

In [101]:
#create an array of the chuncks
chunk_list = []
for k,v in output.items():
    chunk_list.append(v)

In [107]:
#create a list we can extract the words from
chunks = []
for i in chunk_list:
    for j in i:
        chunks.append(j[0])

In [118]:
#get words from chucnks
tmp = []
for i in chunks:
    for j in i:
        tmp.append(j)

#list of just the words
words = tmp[::2]

In [120]:
#get polarity scores
chunk_sent = []
for i in words:
    
    #Blob it
    blob = TextBlob(i)

    #Sentiment Analysis - this funciton removes punctuation and tokenizes 
    sent = blob.sentiment

    #add to sentiment list
    chunk_sent.append(sent.polarity)

In [124]:
#ensure the chunks and polarity scores are the same length
print(len(chunk_sent))
print(len(chunks))

2744
2744


In [128]:
chunk_sent_table = pd.DataFrame({'Chunk': chunks, 'Sentiment Score': chunk_sent})

### Final Result

In [126]:
chunk_sent_table

,Chunk,Sentiment Score
0,"(this, DT)",0.00
1,"(a, DT)",0.00
2,"(dinner, NN)",0.00
3,"(the, DT)",0.00
4,"(first, JJ)",0.25
...,...,...
2739,"(This, DT)",0.00
2740,"(something, NN)",0.00
2741,"(any, DT)",0.00
2742,"(the, DT)",0.00


# Extra Credit

>b.	Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see. 

#### Sort by highest positive scores

In [135]:
cst_top = chunk_sent_table.sort_values(by='Sentiment Score', ascending=False).drop_duplicates()

cst_top.head(40)

,Chunk,Sentiment Score
85,"(excellent, JJ)",1.0
1097,"(best, JJS)",1.0
116,"(awesome.., JJ)",1.0
12,"(Excellent, JJ)",1.0
2316,"(priceless, NN)",1.0
1426,"(brilliant, JJ)",0.9
678,"(brilliant, NN)",0.9
252,"(intelligent, NN)",0.8
25,"(great, JJ)",0.8
24,"(Great, JJ)",0.8


#### Sort by lowest scores

In [136]:
cst_low = chunk_sent_table.sort_values(by='Sentiment Score', ascending=True).drop_duplicates()
cst_low.head(40)

,Chunk,Sentiment Score
2012,"(Worst, NNP)",-1.000000
585,"(outrageous, JJ)",-1.000000
214,"(boring, JJ)",-1.000000
926,"(idiots, NNS)",-0.800000
2486,"(annoying, JJ)",-0.800000
15,"(violent, NN)",-0.800000
340,"(hate, NN)",-0.800000
992,"(crap, NN)",-0.800000
2669,"(stupid, JJ)",-0.800000
1725,"(crude, NN)",-0.700000


The polarity scores of the words seems to make sense for the most part.  A couple of interesting observations are some of the words that have high positive scores.  For exapmle, 'own' has a polarity score of 0.6.  That seems rather odd to me as it seems to be a little ambiguous for such a high score.  On the negative side, it seems odd that 'hate' only has a polarit score of -0.8. I would consider 'hate' to be perhaps one of the top words used to convey a negative sentiment.  As the scores get into the -.4's and -.3's it seems like the words become more contextual, which makes sense that they are closer to zero.  For example, 'subtle' is -0.33. I could see how 'subtle' could be used in a positive or negative context.  